In [1]:
import os
from PIL import Image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt


In [2]:

def remove_corrupt_images(folder_path):
    for subdir, _, files in os.walk(folder_path):
        for file in files:
            file_path = os.path.join(subdir, file)
            try:
                img = Image.open(file_path)
                img.verify()
            except Exception as e:
                print(f"Removing corrupt image: {file_path}")
                os.remove(file_path)


remove_corrupt_images('C:/DoctorP_dataset_split')


In [3]:

base_dir = 'C:/DoctorP_dataset_split'
train_path = os.path.join(base_dir, 'train')
val_path = os.path.join(base_dir, 'val')
test_path = os.path.join(base_dir, 'test')


image_size = (224, 224)
batch_size = 32

train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)


train_gen = train_datagen.flow_from_directory(train_path, target_size=image_size, batch_size=batch_size, class_mode='categorical')
val_gen = val_datagen.flow_from_directory(val_path, target_size=image_size, batch_size=batch_size, class_mode='categorical')
test_gen = val_datagen.flow_from_directory(test_path, target_size=image_size, batch_size=batch_size, class_mode='categorical', shuffle=False)


Found 31158 images belonging to 31 classes.
Found 6682 images belonging to 31 classes.
Found 6696 images belonging to 31 classes.


In [4]:

base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  

model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(128, activation='relu'),
    Dense(train_gen.num_classes, activation='softmax')  
])


model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenetv2_1.00_224 (Funct  (None, 7, 7, 1280)       2257984   
 ional)                                                          
                                                                 
 global_average_pooling2d (G  (None, 1280)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 128)               163968    
                                                                 
 dense_1 (Dense)             (None, 31)                3999      
                                                                 
Total params: 2,425,951
Trainable params: 167,967
Non-trainable params: 2,257,984
_________________________________________________________________


In [5]:

callbacks = [
    EarlyStopping(patience=5, restore_best_weights=True),
    ModelCheckpoint('plant_disease_model.h5', save_best_only=True)
]

history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=15,
    callbacks=callbacks
)



Epoch 1/15
974/974 [==============================] - 840s 848ms/step - loss: 0.5762 - accuracy: 0.8268 - val_loss: 0.3582 - val_accuracy: 0.8830
Epoch 2/15
974/974 [==============================] - 612s 628ms/step - loss: 0.2876 - accuracy: 0.9036 - val_loss: 0.3140 - val_accuracy: 0.8957
Epoch 3/15
974/974 [==============================] - 628s 645ms/step - loss: 0.2170 - accuracy: 0.9273 - val_loss: 0.2589 - val_accuracy: 0.9145
Epoch 4/15
974/974 [==============================] - 695s 713ms/step - loss: 0.1646 - accuracy: 0.9420 - val_loss: 0.2609 - val_accuracy: 0.9175
Epoch 5/15
974/974 [==============================] - 629s 646ms/step - loss: 0.1425 - accuracy: 0.9490 - val_loss: 0.2509 - val_accuracy: 0.9202
Epoch 6/15
974/974 [==============================] - 622s 639ms/step - loss: 0.1156 - accuracy: 0.9586 - val_loss: 0.3010 - val_accuracy: 0.9135
Epoch 7/15
974/974 [==============================] - 560s 575ms/step - loss: 0.0933 - accuracy: 0.9670 - val_loss: 0.2634 -

In [6]:
loss, acc = model.evaluate(test_gen)
print(f"Test Accuracy: {acc*100:.2f}%")

210/210 [==============================] - 100s 477ms/step - loss: 0.2887 - accuracy: 0.9132
Test Accuracy: 91.32%


In [7]:
model.save("plant_disease_model.h5")

model.save("plant_disease_model_saved")

INFO:tensorflow:Assets written to: plant_disease_model_saved\assets


INFO:tensorflow:Assets written to: plant_disease_model_saved\assets


In [ ]:
import tensorflow as tf
converter = tf.lite.TFLiteConverter.from_saved_model("plant_disease_model_saved")
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()


with open("plant_disease_model.tflite", "wb") as f:
    f.write(tflite_model)

print("Model saved in these formats: plant_disease_model.h5، plant_disease_model_saved،plant_disease_model.tflite")

Model saved in these formats: plant_disease_model.h5، plant_disease_model_saved،plant_disease_model.tflite


In [10]:
import os

TRAIN_PATH="C:/DoctorP_dataset_split/train"
class_names = sorted(os.listdir(TRAIN_PATH))

with open("DISEASES label.txt", "w") as f:
    for class_name in class_names:
        f.write(class_name + "\n")

print("SAVED")

SAVED
